**About** : This notebook is used to prepare the data.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
cd ../src/

### Imports

In [ ]:
import os
import sys
import glob
import json
import torch
import librosa
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm
from copy import deepcopy
from joblib import delayed
from sklearn.metrics import *

warnings.simplefilter(action="ignore", category=UserWarning)
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 200)


In [ ]:
from params import *
from util.plots import *

from data.preparation import *
from data.dataset import WaveDataset, PLDataset
from data.transforms import get_transfos

from util.logger import create_logger, save_config, prepare_log_folder
from util.torch import init_distributed

from model_zoo.models import define_model
from model_zoo.melspec import FeatureExtractor, CustomTimeMasking, CustomFreqMasking

from training.losses import BirdLoss
from training.main import k_fold

### Data

In [ ]:
df = prepare_data_2()
df_xc = prepare_xenocanto_data()
# df = prepare_nocall_data()

df.sample(5)

In [ ]:
# df_ = pd.concat([
#     df[~df["primary_label"].apply(lambda x: isinstance(x, list))],
#     df[df["primary_label"].apply(lambda x: isinstance(x, list))].explode('primary_label')
# ], ignore_index=True)

# cts = df_["primary_label"].value_counts()
# cts = np.array([cts.to_dict()[c] for c in CLASSES])

# ws = (1 / (cts / 500)) ** 0.4

In [ ]:
folders = [
    "../logs/2024-05-14/17/",
    "../logs/2024-05-14/16/",
    "../logs/2024-05-14/15/",
    "../logs/2024-05-14/14/",
    "../logs/2024-05-14/12/",
    "../logs/2024-05-14/8/",
    "../output/cpmp_preds_72/pl_sub.csv",
]

# for f in tqdm(folders):
#     for fold in [0, 1, 2, 3]:
#         s = pd.read_csv(f + f"pl_sub_{fold}.csv")
#         scores = np.load(f + f"pl_preds_{fold}.npy")
#         # np.save(f + f"pl_preds_{fold}.npy", scores)
#         s[["row_id"]].to_csv(f + f"pl_preds_{fold}.csv", index=False)

In [ ]:
# pls = []

# fold = 0 # "fullfit"
# for f in folders:
#     if f.endswith('.csv'):
#         pls.append(f[:-4])
#     elif "fullfit" not in str(fold):
#         pls.append(f + f"pl_preds_{fold}")
#     else:
#         pls += [f[:-4] for f in glob.glob(f + "pl_preds_*.csv")]

In [ ]:
# dataset = PLDataset(pls, normalize="std")
# x, y, y_aux, w = dataset[0]

# pl_loader = torch.utils.data.DataLoader(
#     dataset,
#     batch_size=256,
#     shuffle=True,
#     drop_last=True,
#     num_workers=8,
#     pin_memory=False,
# )
# pl_iterator = iter(pl_loader)

In [ ]:
# for i in tqdm(range(len(pl_loader))):
#     x, y, y_aux, w = next(pl_iterator)

In [ ]:
transforms = get_transfos(augment=True, strength=0)

In [ ]:
dataset = WaveDataset(
    df,
    transforms,
    max_len=32000 * 5,
    train=True,
    self_mixup=True,
    secondary_labels_weight=1,
    normalize="std"
)

In [ ]:
# for i in tqdm(range(len(dataset))):
#     # try:
#     x = dataset[i]
#     # except:
#     #     print(i)
#     #     break

In [ ]:
wave, y, y_aux, w = dataset[np.random.choice(len(dataset))]
wave_2, y_2, y_aux_2, w_2 = dataset[np.random.choice(len(dataset))]
# w = wave
y.sum(), y_aux.sum()

In [ ]:
# display_audio(wave.numpy(), title=", ".join(([CLASSES[int(c)] for c in np.argwhere(y.numpy())[0]])))

In [ ]:
# display_audio(w.numpy(), title=", ".join(([CLASSES[int(c)] for c in np.argwhere(y.numpy())[0]])))

### Spectrograms

In [ ]:
params = {
    "sample_rate": 32000,
    "n_mels": 224,  # 128, 224
    "f_min": 90,  # 50
    "f_max": 14000,  # 15000
    "n_fft": 1536,  # 1536
    "hop_length": 717,  # 717
    "win_length": 1024,
    "mel_scale": "htk",
    "power": 2.0,
}

aug_config = {
    "specaug_freq": {
        "mask_max_length": 10,
        "mask_max_masks": 3,
        "p": 0.,
    },
    "specaug_time": {
        "mask_max_length": 20,
        "mask_max_masks": 3,
        "p": 0.,
    },
    "mixup":
    {
        "p_audio": 1,
        "p_spec": 0,
        "additive": True,
        "alpha": 4,
        "num_classes": 182,
        "norm": "std",
    }
}

In [ ]:
ft_extractor = FeatureExtractor(params, aug_config=aug_config, norm="simple")
ft_extractor = ft_extractor.train()

In [ ]:
x = torch.stack([wave, wave_2])
if len(y.size()) < 2:
    y = torch.stack([y, y_2])

In [ ]:
with torch.cuda.amp.autocast(enabled=True):
    melspec, _, _, _ = ft_extractor(x)

In [ ]:
melspec.size()

In [ ]:
plot_spectrogram(melspec[0].numpy(), params)

In [ ]:
# plot_spectrogram(melspec[1].numpy(), params)

### Model

In [ ]:
# "mixnet_s" "mobilenetv2_100" "mnasnet_100" "tf_efficientnet_b0" "tinynet_b"

In [ ]:
model = define_model(
    "efficientvit_b0", 
    params,
    head="gem",
    n_channels=3,
    num_classes=NUM_CLASSES,
)

In [ ]:
from util.torch import count_parameters
count_parameters(model)

In [ ]:
pred, _, _, w = model(x, y, y_aux, w)

In [ ]:
y.size(), pred.size()

In [ ]:
loss = BirdLoss({"name": "bce", "top_k": 0, "ousm_k": 0})
loss(pred, y)

### Train

In [ ]:
class Config:
    """
    Parameters used for training
    """
    # General
    seed = 42
    verbose = 1
    device = "cuda"
    save_weights = True

    # Data
    use_nocall = False
    upsample_low_freq = False

    train_duration = 5  # 15, 5
    duration = 5
    random_crop = False

    aug_strength = 1
    self_mixup = True
    normalize = True  # False ??
    exportable = False

    melspec_config = {
        "sample_rate": 32000,
        "n_mels": 128,  # 128, 224
        "f_min": 50,
        "f_max": 15000,
        "n_fft": 2048,
        "hop_length": 512,  # 716, 512, 417
        "normalized": True,
    }

    aug_config = {
        "specaug_freq": {
            "mask_max_length": 10,
            "mask_max_masks": 3,
            "p": 0.5,
        },
        "specaug_time": {
            "mask_max_length": 20,
            "mask_max_masks": 3,
            "p": 0.5,
        },
        "mixup":
        {
            "p_audio": 0.5,
            "p_spec": 0.2,
            "additive": True,
            "alpha": 4,
            "num_classes": 182,
        }
    }

    # k-fold
    k = 4
    folds_file = f"../input/folds_{k}.csv"
    selected_folds = [0, 1, 2, 3]

    # Model
    name = "tf_efficientnetv2_s"  # tf_efficientnetv2_s maxvit_tiny_tf_384 eca_nfnet_l0
    pretrained_weights = None

    num_classes = 182
    drop_rate = 0.2
    drop_path_rate = 0.2
    n_channels = 1
    head = "gem"
    reduce_stride = "256" in name

    # Training
    loss_config = {
        "name": "focal_bce",
        "weighted": False,
        "smoothing": 0.,
        "top_k": 0,
        "ousm_k": 0,
        "activation": "sigmoid",  # "softmax"
    }
    secondary_labels_weight = 0.5 if loss_config["name"] == "ce" else 1.

    data_config = {
        "batch_size": 8,
        "val_bs": 256,
        "num_classes": num_classes,
        "num_workers": 8,
    }

    optimizer_config = {
        "name": "Ranger",
        "lr": 5e-3,
        "warmup_prop": 0.0,
        "betas": (0.9, 0.999),
        "max_grad_norm": 0.1,
        "weight_decay": 0.,
    }

    epochs = 1

    use_fp16 = True
    verbose = 1
    verbose_eval = 20

    fullfit = True
    n_fullfit = 1

In [ ]:
DEBUG = True

In [ ]:
log_folder = None
config = Config

if not DEBUG:
    log_folder = prepare_log_folder(LOG_PATH)
    print(f'\n -> Logging results to {log_folder}\n')

    create_logger(directory=log_folder, name="logs.txt")
    save_config(config, log_folder + "config.json")

init_distributed(config)

In [ ]:
df = prepare_data(DATA_PATH)
# df_xc = prepare_xenocanto_data(DATA_PATH)
# df = pd.concat([df, df_xc], ignore_index=True)

# if DEBUG:
#     df = df.sample(100).reset_index(drop=True)

In [ ]:
k_fold(config, df, log_folder=log_folder)

Done !